In [1]:
import accelerate
import datasets
import transformers
from transformers import AutoTokenizer
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["XRT_TPU_CONFIG"] = "localservice;0;localhost:51011" 
import datasets
from transformers import EncoderDecoderModel
from datasets import load_dataset

from transformers import BertConfig, EncoderDecoderConfig,EncoderDecoderModel
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional
import json
from pathlib import Path
from typing import Optional, Tuple, Dict
import joblib
from transformers import PreTrainedTokenizer
from tqdm import tqdm
os.environ['HF_DATASETS_CACHE'] = "~/temp/cache"

2022-07-28 17:58:42.541839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [55]:
from transformers import AutoConfig, AutoModel

In [2]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

In [3]:
#######load tokenizer
print("Load Data")

full_data = datasets.load_from_disk("/home/thienchuaheadshot_gmail_com/spin_content/data_clean")

Load Data


In [4]:
full_data[0]

{'category': 'Giày búp bê mũi tròn',
 'name': 'GIẦY BÚP BÊ NỮ ALDO BRAYLYNN',
 'brand': 'ALDO',
 'text': 'thương hiệu thời trang aldo là hãng thời trang canada chuyên mang đến những thiết kế sang trọng và tinh tế về phụ kiện giầy dép và túi xách. aldo được phân phối độc quyền tại việt nam bởi công ty cổ phần nhà thái từ năm 2009. túi và phụ kiện aldo không có hộp và túi vải. cần hỗ trợ tư vấn, vui lòng inbox nhân viên cskh của aldo.',
 'id': 0}

In [5]:
# #Load Tokenizer
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt2-vietnamese")
# model = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt2-vietnamese")

### Configurations

In [45]:
UNFREEZE_LAST_N = 6


MODEL = "NlpHUST/gpt2-vietnamese"
SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
TRAIN_SIZE      = 0.8
TRAIN_BATCHSIZE = 32
BATCH_UPDATE    = 64


MAXLEN          = 768    #{768, 1024, 1280, 1600}
EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1000

SEED            = 2020

In [7]:
import random
import numpy as np
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [8]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        #model = AutoModel.from_pretrained(load_model_path)
        model.load_state_dict(torch.load(load_model_path))

    return model

In [9]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Special tokens added


In [10]:
model.config

GPT2Config {
  "_name_or_path": "NlpHUST/gpt2-vietnamese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50258,
  "embd_pdrop": 0.0,
  "eos_token_id": 50259,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50261,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "sep_token_id": 50262,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50263
}

# Data and loaders

In [12]:
batch = [{"text":"Đây là sản phẩm giày cao gót đi vừa êm vừa sướng, đi rất là tôn giáng đó!", "name": 'GIẦY BÚP BÊ NỮ ALDO BRAYLYNN'},
     {"text":"Giày này không bán em ơi!", "name": 'GIẦY BÚP BÊ NỮ NVIDIA'}]
names = [x['name'] for x in batch]
texts = [x['text'] for x in batch]

In [25]:
inputs = tokenizer([ SPECIAL_TOKENS['bos_token'] + name + SPECIAL_TOKENS['sep_token'] + text + SPECIAL_TOKENS['eos_token'] for name,text in zip(names,texts)], padding = "max_length", truncation=True, max_length=MAXLEN,return_tensors="np")

In [29]:
inputs['input_ids']

array([[50258, 10858, 23487, ..., 50261, 50261, 50261],
       [50258, 10858, 23487, ..., 50261, 50261, 50261]])

In [18]:
encodings_dict = tokenizer(input,                                   
                            truncation=True, 
                            max_length=MAXLEN, 
                            padding="max_length") 

In [22]:
encodings_dict

{'input_ids': [50258, 10858, 23487, 373, 29662, 39552, 20662, 12771, 19251, 8759, 5626, 28579, 3933, 50262, 2424, 314, 578, 659, 2664, 592, 6063, 585, 959, 3751, 959, 4056, 16, 585, 595, 314, 1971, 6658, 467, 5, 50259, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 50261, 5026

In [30]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]  
    names = batch['name']
    texts = batch['text']
    inputs = tokenizer([ SPECIAL_TOKENS['bos_token'] + name + SPECIAL_TOKENS['sep_token'] + text + SPECIAL_TOKENS['eos_token'] for name,text in zip(names,texts)], padding = "max_length", truncation=True, max_length=MAXLEN,return_tensors="np")
    batch["labels"] = inputs.input_ids 
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                                                                                                           
    return batch


In [31]:
full_data = full_data.filter(lambda x: len(x["text"]) >8 )

Loading cached processed dataset at /home/thienchuaheadshot_gmail_com/spin_content/data_clean/cache-ab795d76a411af75.arrow


In [32]:
full_data

Dataset({
    features: ['category', 'name', 'brand', 'text', 'id'],
    num_rows: 40554
})

In [33]:
train_data = full_data.map(
    process_data_to_model_inputs, 
    batched=True, 
#     batch_size=1000, 
    remove_columns=["category", "name",'text','id'],num_proc=10
)

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#4:   0%|          | 0/5 [00:00<?, ?ba/s]

#5:   0%|          | 0/5 [00:00<?, ?ba/s]

#6:   0%|          | 0/5 [00:00<?, ?ba/s]

#7:   0%|          | 0/5 [00:00<?, ?ba/s]

#8:   0%|          | 0/5 [00:00<?, ?ba/s]

#9:   0%|          | 0/5 [00:00<?, ?ba/s]

In [37]:
train_data = train_data.remove_columns(['brand'])

In [38]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

In [42]:
data_full_split = train_data.train_test_split(test_size=0.05, seed = 42)
train_data = data_full_split["train"]                                    
test_data = data_full_split["test"] 

Loading cached split indices for dataset at /home/thienchuaheadshot_gmail_com/spin_content/data_clean/cache-576a95d1249aa1d1.arrow and /home/thienchuaheadshot_gmail_com/spin_content/data_clean/cache-e5519ab616a2556c.arrow


In [44]:
test_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2028
})

In [48]:
%%time

training_args = TrainingArguments(
    output_dir="./",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    prediction_loss_only=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy='steps',
    logging_steps=400,  # set to 1000 for full training
    save_steps=1000,  # set to 500 for full training
    eval_steps=1000,  # set to 8000 for full training
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=3,
    load_best_model_at_end=True,     
)

CPU times: user 4.32 s, sys: 4.14 s, total: 8.47 s
Wall time: 12.6 s


2022-07-28 19:51:28.885462: E tensorflow/core/framework/op_kernel.cc:1693] OpKernel ('op: "TPURoundRobin" device_type: "CPU"') for unknown op: TPURoundRobin
2022-07-28 19:51:28.886266: E tensorflow/core/framework/op_kernel.cc:1693] OpKernel ('op: "TpuHandleToProtoKey" device_type: "CPU"') for unknown op: TpuHandleToProtoKey


In [50]:
# instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()
trainer.save_model()  

***** Running training *****
  Num examples = 38526
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 2048
  Gradient Accumulation steps = 64
  Total optimization steps = 72


Step,Training Loss,Validation Loss


### Generating text with finetune GPT2

In [56]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='./model/pytorch_model.bin')

loading file https://huggingface.co/NlpHUST/gpt2-vietnamese/resolve/main/vocab.json from cache at /home/thienchuaheadshot_gmail_com/.cache/huggingface/transformers/7cf88785b592f4177f364d0c49db894e50b5c1c4f2a4af70dc87fe3ea5296b96.1c1d29c8ea56bae456d84e19ab8005ee46986b2f1cea420bf0ee5b6f37bc5447
loading file https://huggingface.co/NlpHUST/gpt2-vietnamese/resolve/main/merges.txt from cache at /home/thienchuaheadshot_gmail_com/.cache/huggingface/transformers/d604f49ddd467aab72674d3a6fe91d48977020205ee65fee37edbc3793c57331.a70df4dd5bfb05f5ad6646b40581db340b74b8771c707fe89ce64b8fbf08d031
loading file https://huggingface.co/NlpHUST/gpt2-vietnamese/resolve/main/tokenizer.json from cache at /home/thienchuaheadshot_gmail_com/.cache/huggingface/transformers/dcc1ad091eed0791d69c3b1013e546d77d35549c8c6d10316917785e2ff0cfad.61e940ecbaf4ec304b3aab441208b8185502600673c69ddc01d444307febb866
loading file https://huggingface.co/NlpHUST/gpt2-vietnamese/resolve/main/added_tokens.json from cache at /home/thi

Special tokens added


loading configuration file https://huggingface.co/NlpHUST/gpt2-vietnamese/resolve/main/config.json from cache at /home/thienchuaheadshot_gmail_com/.cache/huggingface/transformers/797cd60ee07bb6033508bbfe7156425d229228ae730e6661005b3c85fe55dd3b.c7ce1289c7853d140ef65348be24ee2e0577490b793b1e7fcbb88e3b81737f53
Model config GPT2Config {
  "_name_or_path": "NlpHUST/gpt2-vietnamese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50258,
  "embd_pdrop": 0.0,
  "eos_token_id": 50259,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50261,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "sep_token_id": 50262,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "su

In [53]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dro

In [120]:
name = "Giày Quốc Việt: "
des = "Sản phẩm SM05 là chiếc giày sang trọng của hãng Quốc Việt. Với thiết kế thời trang và độc đáo, bạn có thể mang theo bên mình mọi lúc trong những chuyến đi chơi xa hoặc các hoạt động vui nhộn mà không cần lo lắng về vấn đề an toàn sức khỏe khi sử dụng sản phầm này nữa nhé. Mẫu mã đẹp mắt cùng nhiều ưu điểm nổi bật khác được làm từ chất liệu da bò thật 100% đã tạo nên sự cuốn hút khó cưỡng đối với người nhìn ngay lần đầu tiên ngắm nghía mẫu giầy cao cấp chính hiệu quốc việt tại shop chúng tôi."

prompt = SPECIAL_TOKENS['bos_token'] + name + SPECIAL_TOKENS['sep_token'] + des

In [121]:
prompt

'<|BOS|>Giày Quốc Việt: <|SEP|>Sản phẩm SM05 là chiếc giày sang trọng của hãng Quốc Việt. Với thiết kế thời trang và độc đáo, bạn có thể mang theo bên mình mọi lúc trong những chuyến đi chơi xa hoặc các hoạt động vui nhộn mà không cần lo lắng về vấn đề an toàn sức khỏe khi sử dụng sản phầm này nữa nhé. Mẫu mã đẹp mắt cùng nhiều ưu điểm nổi bật khác được làm từ chất liệu da bò thật 100% đã tạo nên sự cuốn hút khó cưỡng đối với người nhìn ngay lần đầu tiên ngắm nghía mẫu giầy cao cấp chính hiệu quốc việt tại shop chúng tôi.'

In [122]:
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

In [123]:
generated

tensor([[50258, 12610,   883,   649,    30,   225, 50262,  4147,   659,  8982,
          3309,   314,  1076,  2664,  1218,   828,   331,  1520,   883,   649,
            18,  1470,   695,   795,   606,   840,   309,  1295,  2039,    16,
           430,   329,   419,   836,   608,   909,   589,   962,  1036,   366,
           392,  2025,   585,  1092,  1353,   743,   338,   876,   535,  1252,
          5349,   481,   359,   675,   565,  2034,   473,   889,   600,   841,
           710,   967,  1334,   456,   603,   515,   578, 14758,   444,  1078,
          1404,    18,  4206,  1692,   805,  1012,   648,   480,  1370,   738,
          1179,  1838,   474,   354,   471,   450,   668,   936,   904,  2647,
          1086,  1600,     9,   417,   707,   588,   508,  2026,  1444,   983,
          4668,   729,   370,   390,  1103,   987,   897,   524,  1071,  3175,
         17372,  1261,  9196,   592,   656,   577,   745,   921,  2607,   491,
          3796,   634,   554,    18]])

In [124]:
model.eval();

In [125]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated,do_sample=True, min_length=20, max_length=300,top_k=30,top_p=0.7,temperature=0.9,repetition_penalty=2.0,num_return_sequences=3)



In [126]:
for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)   
    print("{}: {}\n\n".format(i+1, text))

1: Giày Quốc Việt: Sản phẩm SM05 là chiếc giày sang trọng của hãng Quốc Việt. Với thiết kế thời trang và độc đáo, bạn có thể mang theo bên mình mọi lúc trong những chuyến đi chơi xa hoặc các hoạt động vui nhộn mà không cần lo lắng về vấn đề an toàn sức khỏe khi sử dụng sản phầm này nữa nhé. Mẫu mã đẹp mắt cùng nhiều ưu điểm nổi bật khác được làm từ chất liệu da bò thật 100% đã tạo nên sự cuốn hút khó cưỡng đối với người nhìn ngay lần đầu tiên ngắm nghía mẫu giầy cao cấp chính hiệu quốc việt tại shop chúng tôi.
Với kiểu dáng đơn giản nhưng lại vô cũng hiện đại phù hợp cho cả nam lẫn nữ. Sản Phẩm Giày Thể Thao Nữ Thời Trang Chính Hãng sẽ giúp ích rất lớn để bảo vệ đôi chân luôn thoải mái nhất. Bạn chỉ mất khoảng 1 giờ đồng hồ mỗi ngày là hoàn thành việc mua sắm đồ dùng gia đình rồi đấy!Một số công trình giao thông trên địa bàn TP Đà Nẵng (TPĐN) do Ban quản lý dự án đường bộ Thăng Long - Sông Hàn thực tế chưa triển khai thi công đúng tiến độ như cam kết ban đâu Tư nhân hoá tuyến tránh Hải